In [2]:
import pandas as pd
import re

# File path
file_path = 'AdjacentCounties'

# Reading the file
with open(file_path, 'r') as file:
    data = file.read()

# Initialize an empty list to store the processed data
county_data = []
current_county = None
current_code = None

# Process each line in the file data
for line in data.strip().splitlines():
    line = line.strip()
    if not line:
        continue

    # Match for main county lines (non-indented) using regex for pattern consistency
    match_main = re.match(r'"(.+?), IN"\s+(\d+)\s+".+?"\s+(\d+)', line)
    if match_main:
        # Extract the main county details only if it is an Indiana county
        current_county = match_main.group(1)
        current_code = match_main.group(2)
    else:
        # Process indented lines for adjacent counties
        match_adjacent = re.match(r'"\s*(.+?)"\s+(\d+)', line)
        if match_adjacent and current_county and current_code:
            adjacent_county = match_adjacent.group(1)
            adjacent_code = match_adjacent.group(2)
            county_data.append([current_county, current_code, adjacent_county, adjacent_code])

# Create DataFrame with Indiana counties only
df = pd.DataFrame(county_data, columns=["County", "County_Code", "Adjacent_County", "Adjacent_County_Code"])

# Get DataFrame with Indiana counties only in the Adjacent_County column
df = df[df["Adjacent_County"].str.endswith(", IN")]
df["Adjacent_County"] = df["Adjacent_County"].str.replace(", IN", "")

In [3]:
adjacent = df

In [4]:
adjacent.head()

,County,County_Code,Adjacent_County,Adjacent_County_Code
0,Adams County,18001,Allen County,18003
1,Adams County,18001,Jay County,18075
2,Adams County,18001,Wells County,18179
5,Allen County,18003,Allen County,18003
6,Allen County,18003,DeKalb County,18033


In [5]:
# Load populations
# Reading the data from the specified file 'county_data.csv'

# File path
file_path = 'county_data.csv'

# Reading the data from the file
pop_df = pd.read_csv(file_path)

# Add ' County' to end of name column
pop_df['name'] = pop_df['name'] + ' County'

pop_df.head()

,id,name,name_ascii,type,county_fips,state_id,state_name,lat,lng,population,metadata_tag
0,1,Whitley County,Whitley,County,18183,IN,Indiana,41.1394,-85.5051,34048,0
1,2,White County,White,County,18181,IN,Indiana,40.7498,-86.8655,24593,0
2,3,Wells County,Wells,County,18179,IN,Indiana,40.7292,-85.2212,28103,0
3,4,Wayne County,Wayne,County,18177,IN,Indiana,39.8644,-85.0098,66588,0
4,5,Washington County,Washington,County,18175,IN,Indiana,38.6000,-86.1053,28025,0


In [6]:
# Sum of all populations divide by 9 (number of counties in Indiana)
ideal_population = pop_df['population'].sum() / 9

# Calculate the min and max based on 5% tolerance
min_population = ideal_population - (ideal_population * 0.1)
max_population = ideal_population + (ideal_population * 0.1)

ideal_population, min_population, max_population

(np.float64(750148.8888888889),
 np.float64(675134.0),
 np.float64(825163.7777777778))

In [7]:
import pandas as pd

# Merge to add main county data (lat, lng, population)
adjacent = pd.merge(
    adjacent, 
    pop_df[['name', 'lat', 'lng', 'population']], 
    left_on='County', 
    right_on='name', 
    how='left'
).rename(columns={'lat': 'main_lat', 'lng': 'main_lng', 'population': 'main_population'})

# Drop the extra 'name' column
adjacent = adjacent.drop(columns=['name'])

# Merge again to add adjacent county data (lat, lng, population)
adjacent = pd.merge(
    adjacent, 
    pop_df[['name', 'lat', 'lng', 'population']], 
    left_on='Adjacent_County', 
    right_on='name', 
    how='left'
).rename(columns={'lat': 'adj_lat', 'lng': 'adj_lng', 'population': 'adj_population'})

# Drop the extra 'name' column from the second merge
adjacent = adjacent.drop(columns=['name'])

# Display the updated DataFrame
adjacent.head()


,County,County_Code,Adjacent_County,Adjacent_County_Code,main_lat,main_lng,main_population,adj_lat,adj_lng,adj_population
0,Adams County,18001,Allen County,18003,40.7457,-84.9366,35685,41.0909,-85.0666,381839
1,Adams County,18001,Jay County,18075,40.7457,-84.9366,35685,40.4380,-85.0057,20570
2,Adams County,18001,Wells County,18179,40.7457,-84.9366,35685,40.7292,-85.2212,28103
3,Allen County,18003,Allen County,18003,41.0909,-85.0666,381839,41.0909,-85.0666,381839
4,Allen County,18003,DeKalb County,18033,41.0909,-85.0666,381839,41.3976,-84.9991,43059


In [8]:
from math import pi, sin, cos, asin, sqrt

def degrees_to_radians(x):
     return((pi/180)*x)
     
def lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b):
    radius_of_earth = 24872/(2*pi)
    c = sin((degrees_to_radians(lat_a) - \
    degrees_to_radians(lat_b))/2)**2 + \
    cos(degrees_to_radians(lat_a)) * \
    cos(degrees_to_radians(lat_b)) * \
    sin((degrees_to_radians(lon_a) - \
    degrees_to_radians(lon_b))/2)**2
    return(2 * radius_of_earth * (asin(sqrt(c))))    

def lon_lat_distance_meters (lon_a,lat_a,lon_b,lat_b):
    return(lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b) * 1609.34) 
    

adjacent['distance_miles'] = adjacent.apply(
    lambda row: lon_lat_distance_miles(row['main_lat'], row['main_lng'], row['adj_lat'], row['adj_lng']),
    axis=1
)

In [9]:
adjacent.head()

,County,County_Code,Adjacent_County,Adjacent_County_Code,main_lat,main_lng,main_population,adj_lat,adj_lng,adj_population,distance_miles
0,Adams County,18001,Allen County,18003,40.7457,-84.9366,35685,41.0909,-85.0666,381839,9.218758
1,Adams County,18001,Jay County,18075,40.7457,-84.9366,35685,40.4380,-85.0057,20570,5.124827
2,Adams County,18001,Wells County,18179,40.7457,-84.9366,35685,40.7292,-85.2212,28103,19.662941
3,Allen County,18003,Allen County,18003,41.0909,-85.0666,381839,41.0909,-85.0666,381839,0.000000
4,Allen County,18003,DeKalb County,18033,41.0909,-85.0666,381839,41.3976,-84.9991,43059,5.011402


In [10]:
county_pop = adjacent[['County_Code', 'main_population']]
county_pop = county_pop.set_index('County_Code')['main_population'].to_dict()
county_pop

{'18001': 35685,
 '18003': 381839,
 '18005': 81759,
 '18007': 8687,
 '18009': 12139,
 '18011': 69839,
 '18013': 15444,
 '18015': 20288,
 '18017': 37918,
 '18019': 120185,
 '18021': 26397,
 '18023': 33010,
 '18025': 10511,
 '18027': 33281,
 '18029': 50494,
 '18031': 26466,
 '18033': 43059,
 '18035': 112480,
 '18037': 43474,
 '18039': 206314,
 '18041': 23393,
 '18043': 79594,
 '18045': 16422,
 '18047': 22769,
 '18049': 20400,
 '18051': 33017,
 '18053': 66802,
 '18055': 30924,
 '18057': 341616,
 '18059': 78616,
 '18061': 39516,
 '18063': 172100,
 '18065': 48857,
 '18067': 83349,
 '18069': 36572,
 '18071': 45948,
 '18073': 33006,
 '18075': 20570,
 '18077': 33000,
 '18079': 27619,
 '18081': 159739,
 '18083': 36362,
 '18085': 80151,
 '18087': 40085,
 '18089': 495925,
 '18091': 112184,
 '18093': 45133,
 '18095': 130037,
 '18097': 969542,
 '18099': 46175,
 '18101': 9885,
 '18103': 36100,
 '18105': 140189,
 '18107': 37967,
 '18109': 71394,
 '18111': 13865,
 '18113': 47293,
 '18115': 5931,
 '181

In [11]:
# Initialize an empty dictionary to hold the result
adjacency_dict = {}

# Iterate over each row in the DataFrame
for _, row in adjacent.iterrows():
    # Extract main county information
    main_county = row['County_Code']
    
    # Calculate distance (assuming you have a function to do this)
    distance = lon_lat_distance_miles(row['main_lng'], row['main_lat'], row['adj_lng'], row['adj_lat'])
    
    # Structure for each adjacent county
    adjacent_info = {
        'Adjacent_County_Code': row['Adjacent_County_Code'],
        'Distance': distance
    }
    
    # If the main county is not yet a key in the dictionary, add it
    if main_county not in adjacency_dict:
        adjacency_dict[main_county] = []
    
    # Append the adjacent county info to the list for this county
    adjacency_dict[main_county].append(adjacent_info)

# Display the adjacency dictionary
adjacency_dict

{'18001': [{'Adjacent_County_Code': '18003', 'Distance': 24.796349193706693},
  {'Adjacent_County_Code': '18075', 'Distance': 21.565539067531635},
  {'Adjacent_County_Code': '18179', 'Distance': 14.9421243660823}],
 '18003': [{'Adjacent_County_Code': '18003', 'Distance': 0.0},
  {'Adjacent_County_Code': '18033', 'Distance': 21.477734667532314},
  {'Adjacent_County_Code': '18069', 'Distance': 28.46955246430767},
  {'Adjacent_County_Code': '18113', 'Distance': 28.00369041650233},
  {'Adjacent_County_Code': '18179', 'Distance': 26.260837334188672},
  {'Adjacent_County_Code': '18183', 'Distance': 23.06892348165851}],
 '18005': [{'Adjacent_County_Code': '18013', 'Distance': 17.67019988610346},
  {'Adjacent_County_Code': '18031', 'Distance': 22.329820068143857},
  {'Adjacent_County_Code': '18071', 'Distance': 22.017787257580256},
  {'Adjacent_County_Code': '18079', 'Distance': 20.436152828816397},
  {'Adjacent_County_Code': '18081', 'Distance': 22.445119008847804},
  {'Adjacent_County_Code':

# Build Linear Programming Problem



Our objective is to maximize compactness (minimize distance between counties in the same district). Our constraints are:

• Each county must be in exactly one district.  
• The population of each district must be within 15% of the ideal population (the mean of the total population divided by the number of districts). These are currently stored in ideal_population, min_population, max_population variables.  
• The number of districts must be 9 (the current number of districts in Indiana).  
• Counties in the same district must be adjacent.

In [64]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpBinary, LpStatus, PULP_CBC_CMD

# Function to solve the problem with given tolerance
def solve_districting_problem(adjacency_dict, county_pop, target_pop, initial_tolerance=0.05, max_tolerance=0.20, tolerance_step=0.01):
    # Start with the initial tolerance for population balance
    tolerance = initial_tolerance
    
    while tolerance <= max_tolerance:
        print(f"Trying with population tolerance: {tolerance*100:.1f}%")
        
        # Define population bounds based on the current tolerance
        min_pop = target_pop * (1 - tolerance)
        max_pop = target_pop * (1 + tolerance)
        
        # Create the model
        prob = LpProblem("Indiana Districting", LpMinimize)

        # Number of districts
        num_districts = 9

        # Create binary variables for assigning counties to districts
        x = LpVariable.dicts("x", 
                             ((i, d) for i in county_pop.keys() for d in range(1, num_districts + 1)), 
                             0, 1, 
                             LpBinary)

        # Auxiliary variables for compactness measure (distance minimization)
        y = LpVariable.dicts("y", 
                             ((county, adj['Adjacent_County_Code'], d) for county in adjacency_dict
                              for adj in adjacency_dict[county] for d in range(1, num_districts + 1)), 
                             0, 1, 
                             LpBinary)

        # Objective function: minimize total distance within districts for compactness
        prob += lpSum(y[county, adj['Adjacent_County_Code'], d] * adj['Distance']
                      for county in adjacency_dict
                      for adj in adjacency_dict[county]
                      for d in range(1, num_districts + 1))

        # Constraint 1: each county must be in exactly one district
        for county in county_pop.keys():
            prob += lpSum(x[county, d] for d in range(1, num_districts + 1)) == 1

        # Constraint 2: population balance
        for d in range(1, num_districts + 1):
            district_pop = lpSum(x[county, d] * county_pop[county] for county in county_pop.keys())
            prob += district_pop >= min_pop
            prob += district_pop <= max_pop

        # Constraint 3: adjacency and auxiliary variable constraints
        for county in adjacency_dict:
            for adj in adjacency_dict[county]:
                adj_county = adj['Adjacent_County_Code']
                for d in range(1, num_districts + 1):
                    # Enforce y[(county, adj_county, d)] = x[county, d] * x[adj_county, d]
                    prob += y[(county, adj_county, d)] <= x[county, d]
                    prob += y[(county, adj_county, d)] <= x[adj_county, d]
                    prob += y[(county, adj_county, d)] >= x[county, d] + x[adj_county, d] - 1

        # Solve the problem
        solver = PULP_CBC_CMD(msg=1)
        status = prob.solve(solver)

        # Check if the solution is feasible
        if LpStatus[prob.status] == "Optimal":
            print("\nSolution Status:", LpStatus[prob.status])
            results = []

            print("\nDistrict Assignments:")
            for d in range(1, num_districts + 1):
                district_counties = []
                district_pop = 0
                for county in county_pop.keys():
                    if x[county, d].value() > 0.5:
                        district_counties.append(county)
                        district_pop += county_pop[county]

                        results.append({
                            "County": county,
                            "District": d,
                            "Population": county_pop[county]
                        })
                print(f"\nDistrict {d}:")
                print(f"Counties: {', '.join(str(c) for c in district_counties)}")
                print(f"Population: {district_pop:,} ({district_pop/target_pop*100:.1f}% of target)")
                
                # Save to file
                results_df = pd.DataFrame(results)
                results_df.to_csv("district_assignments.csv", index=False)

            return results  # Return results if feasible solution is found
        else:
            print("Model infeasible at tolerance:", tolerance)
            tolerance += tolerance_step  # Increase tolerance and try again

    print("No feasible solution found within tolerance limits.")
    return None

# Define initial parameters for population tolerance
initial_tolerance = 0.05
max_tolerance = 0.50
tolerance_step = 0.01
num_districts = 9

# Define target population per district
total_pop = sum(county_pop.values())
target_pop = total_pop / num_districts

# Run the districting problem with varying tolerances
results = solve_districting_problem(adjacency_dict, county_pop, target_pop, initial_tolerance, max_tolerance, tolerance_step)


Trying with population tolerance: 5.0%


/Users/mattmiller/Northwestern/Decision Analytics/Assignment 3/Assignment3_AlgorithmicRedistricting_MSDS460/venv/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mattmiller/Northwestern/Decision Analytics/Assignment 3/Assignment3_AlgorithmicRedistricting_MSDS460/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/7673db4ea41d4cd58dcc2573e4f63606-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/7673db4ea41d4cd58dcc2573e4f63606-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14155 COLUMNS
At line 63485 RHS
At line 77636 BOUNDS
At line 83082 ENDATA
Problem MODEL has 14150 rows, 5445 columns and 34533 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.02 seconds
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to al

In [65]:
# Load the district assignment data
results_df = pd.read_csv("district_assignments.csv")

# Ensure consistent data types for the merge
results_df['County_Code'] = results_df['County'].astype(str)
adjacent['County_Code'] = adjacent['County_Code'].astype(str)

# Create a dictionary to map County_Code to County names
county_name_map = dict(zip(adjacent['County_Code'], adjacent['County']))

# Add the County names to results_df
results_df['County_Name'] = results_df['County_Code'].map(county_name_map)

In [66]:
results_df.head()

,County,District,Population,County_Code,County_Name
0,18007,1,8687,18007,Benton County
1,18043,1,79594,18043,Floyd County
2,18087,1,40085,18087,LaGrange County
3,18101,1,9885,18101,Martin County
4,18103,1,36100,18103,Miami County


In [67]:
import folium
import pandas as pd

# Merge latitude/longitude information from `adjacent` to `results_df` based on `County_Code`
results_df = results_df.merge(adjacent[['County_Code', 'main_lat', 'main_lng']], on='County_Code', how='left')

# Check for missing latitude/longitude values and handle them
missing_location = results_df[results_df[['main_lat', 'main_lng']].isna().any(axis=1)]

if not missing_location.empty:
    print("Warning: The following counties have missing location data and will be assigned default coordinates:")
    print(missing_location[['County_Name', 'District']])
    
    # Assign default coordinates (center of Indiana) to counties with missing lat/lng
    results_df[['main_lat', 'main_lng']] = results_df[['main_lat', 'main_lng']].fillna([40.273502, -86.126976])

# Create a map centered on Indiana
m = folium.Map(location=[40.273502, -86.126976], zoom_start=7)

# Define colors for each district (up to 9 districts)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue']

# Add markers for each county using different colors for each district
for _, row in results_df.iterrows():
    county_name = row['County_Name']
    district = row['District']
    lat = row['main_lat']
    lng = row['main_lng']
    color = colors[district - 1]  # Use district number to pick a color from the list

    # Add a marker for each county
    folium.Marker(
        [lat, lng],
        popup=f"{county_name} (District {district})",
        icon=folium.Icon(color=color)
    ).add_to(m)

# Save the map to an HTML file
m.save("indiana_districts.html")
m

# Initial Conclusion

The constraints worked to create a group of counties with somewhat similar populations but only worked if the tolerance of those population balances were 30% or higher. It also did not group them next to each other even though that constraint was added. We did not have the adjacency as a strict constraint (or with a penalty) but I will try that below now.

In [68]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpBinary, LpStatus, PULP_CBC_CMD

# Function to solve the problem with given tolerance
def solve_districting_problem(adjacency_dict, county_pop, target_pop, initial_tolerance=0.05, max_tolerance=0.20, tolerance_step=0.01):
    tolerance = initial_tolerance
    while tolerance <= max_tolerance:
        print(f"Trying with population tolerance: {tolerance*100:.1f}%")
        
        # Define population bounds
        min_pop = target_pop * (1 - tolerance)
        max_pop = target_pop * (1 + tolerance)
        
        # Create the model
        prob = LpProblem("Indiana Districting", LpMinimize)

        # Number of districts
        num_districts = 9

        # Create binary variables for assigning counties to districts
        x = LpVariable.dicts("x", 
                             ((i, d) for i in county_pop.keys() for d in range(1, num_districts + 1)), 
                             0, 1, 
                             LpBinary)

        # Auxiliary variables for compactness measure (distance minimization)
        y = LpVariable.dicts("y", 
                             ((county, adj['Adjacent_County_Code'], d) for county in adjacency_dict
                              for adj in adjacency_dict[county] for d in range(1, num_districts + 1)), 
                             0, 1, 
                             LpBinary)

        # Create auxiliary variables for population deviation in each district
        deviation = LpVariable.dicts("deviation", range(1, num_districts + 1), 0)

        # Objective function: minimize total population deviation
        prob += lpSum(deviation[d] for d in range(1, num_districts + 1))

        # Constraint 1: each county must be in exactly one district
        for county in county_pop.keys():
            prob += lpSum(x[county, d] for d in range(1, num_districts + 1)) == 1

        # Constraint 2: population balance with deviation
        for d in range(1, num_districts + 1):
            district_pop = lpSum(x[county, d] * county_pop[county] for county in county_pop.keys())
            prob += district_pop - target_pop <= deviation[d]
            prob += target_pop - district_pop <= deviation[d]

        # Constraint 3: strict adjacency enforcement
        for county in adjacency_dict:
            for d in range(1, num_districts + 1):
                # A county can only be in district `d` if at least one adjacent county is also in district `d`
                prob += x[county, d] <= lpSum(x[adj['Adjacent_County_Code'], d] for adj in adjacency_dict[county])

        # Solve the problem
        solver = PULP_CBC_CMD(msg=1)
        status = prob.solve(solver)

        # Check if the solution is feasible
        if LpStatus[prob.status] == "Optimal":
            print("\nSolution Status:", LpStatus[prob.status])
            results = []

            print("\nDistrict Assignments:")
            for d in range(1, num_districts + 1):
                district_counties = []
                district_pop = 0
                for county in county_pop.keys():
                    if x[county, d].value() > 0.5:
                        district_counties.append(county)
                        district_pop += county_pop[county]

                        results.append({
                            "County": county,
                            "District": d,
                            "Population": county_pop[county]
                        })
                print(f"\nDistrict {d}:")
                print(f"Counties: {', '.join(str(c) for c in district_counties)}")
                print(f"Population: {district_pop:,} ({district_pop/target_pop*100:.1f}% of target)")
                
                # Save to file
                results_df = pd.DataFrame(results)
                results_df.to_csv("district_assignments.csv", index=False)

            return results  # Return results if feasible solution is found
        else:
            print("Model infeasible at tolerance:", tolerance)
            tolerance += tolerance_step  # Increase tolerance and try again

    print("No feasible solution found within tolerance limits.")
    return None

# Define initial parameters for population tolerance
initial_tolerance = 0.05
max_tolerance = 0.50
tolerance_step = 0.01
num_districts = 9

# Define target population per district
total_pop = sum(county_pop.values())
target_pop = total_pop / num_districts

# Run the districting problem with varying tolerances
results = solve_districting_problem(adjacency_dict, county_pop, target_pop, initial_tolerance, max_tolerance, tolerance_step)


Trying with population tolerance: 5.0%


/Users/mattmiller/Northwestern/Decision Analytics/Assignment 3/Assignment3_AlgorithmicRedistricting_MSDS460/venv/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mattmiller/Northwestern/Decision Analytics/Assignment 3/Assignment3_AlgorithmicRedistricting_MSDS460/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/4d6ac1c63a7942258e1124390c99d80e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/4d6ac1c63a7942258e1124390c99d80e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 943 COLUMNS
At line 9845 RHS
At line 10784 BOUNDS
At line 11613 ENDATA
Problem MODEL has 938 rows, 837 columns and 6525 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 92 SOS with 828 members
Cgl0004I processed model has 227 rows, 837 columns (828 integer (828 of which binary)) and 3168 elements
Cbc0038I Initial state - 17 

In [69]:
# Load the district assignment data
results_df = pd.read_csv("district_assignments.csv")

# Ensure consistent data types for the merge
results_df['County_Code'] = results_df['County'].astype(str)
adjacent['County_Code'] = adjacent['County_Code'].astype(str)

# Create a dictionary to map County_Code to County names
county_name_map = dict(zip(adjacent['County_Code'], adjacent['County']))

# Add the County names to results_df
results_df['County_Name'] = results_df['County_Code'].map(county_name_map)

In [70]:
results_df.head()

,County,District,Population,County_Code,County_Name
0,18041,1,23393,18041,Fayette County
1,18089,1,495925,18089,Lake County
2,18103,1,36100,18103,Miami County
3,18105,1,140189,18105,Monroe County
4,18129,1,25301,18129,Posey County


In [71]:
import folium
import pandas as pd

# Merge latitude/longitude information from `adjacent` to `results_df` based on `County_Code`
results_df = results_df.merge(adjacent[['County_Code', 'main_lat', 'main_lng']], on='County_Code', how='left')

# Check for missing latitude/longitude values and handle them
missing_location = results_df[results_df[['main_lat', 'main_lng']].isna().any(axis=1)]

if not missing_location.empty:
    print("Warning: The following counties have missing location data and will be assigned default coordinates:")
    print(missing_location[['County_Name', 'District']])
    
    # Assign default coordinates (center of Indiana) to counties with missing lat/lng
    results_df[['main_lat', 'main_lng']] = results_df[['main_lat', 'main_lng']].fillna([40.273502, -86.126976])

# Create a map centered on Indiana
m = folium.Map(location=[40.273502, -86.126976], zoom_start=7)

# Define colors for each district (up to 9 districts)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue']

# Add markers for each county using different colors for each district
for _, row in results_df.iterrows():
    county_name = row['County_Name']
    district = row['District']
    lat = row['main_lat']
    lng = row['main_lng']
    color = colors[district - 1]  # Use district number to pick a color from the list

    # Add a marker for each county
    folium.Marker(
        [lat, lng],
        popup=f"{county_name} (District {district})",
        icon=folium.Icon(color=color)
    ).add_to(m)

# Save the map to an HTML file
m.save("indiana_districts.html")
m

# Additional Conclusion

The adjacency constraints worked a little better this time to put counties next to each other and it actually allowed it with a lower tolerance (5%) but did not completely keep them next to each other. You can also see in district 8, that a lot of the counties are next to each other but they almost draw a line through the map which is not an desired outcome when drawing district lines.

My next line of thinking is that we need to draw the districts, find the centroids and then add constraints for those centroid distances to be minimized. 